# Python 1 - Core Data Analysis - Pandas Package

Below are snippets of code and topics taught in Training The Street Python 1 class.

## Importing Data

In [27]:
#%%% Importing Data
#data can be imported from Excel or CSV files

import pandas as pd
aapl = pd.read_csv('StockData/AAPL.csv')
trades = pd.read_excel('ExData/client_trades.xlsx')
trades = pd.read_excel('ExData/client_trades.xlsx', sheet_name=1)

## Exploring Data

In [28]:
#%%% Exploring Data
aapl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9744 entries, 0 to 9743
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       9744 non-null   object 
 1   Open       9743 non-null   float64
 2   High       9743 non-null   float64
 3   Low        9743 non-null   float64
 4   Close      9743 non-null   float64
 5   Adj Close  9743 non-null   float64
 6   Volume     9743 non-null   float64
dtypes: float64(6), object(1)
memory usage: 533.0+ KB


In [29]:
trades.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      98 non-null     object
 1   ticker    98 non-null     object
 2   buy_sell  98 non-null     object
 3   quantity  98 non-null     int64 
dtypes: int64(1), object(3)
memory usage: 3.2+ KB


In [30]:
trades['ticker'].value_counts()

WHR     5
NEM     5
PG      4
INTC    4
JNPR    4
AVGO    4
TRIP    3
JWN     3
EXPE    3
ADBE    3
QCOM    3
BAC     3
GIS     3
GLW     3
GM      2
CTXS    2
TAP     2
TXN     2
TJX     2
FLT     2
CSCO    2
C       2
NKE     2
HD      2
ADI     2
EA      2
CRM     2
NBL     2
LRCX    2
MSI     2
BBY     2
WYNN    2
PEP     1
NVDA    1
WDC     1
MA      1
COST    1
MSFT    1
V       1
AMD     1
T       1
JPM     1
DG      1
CMG     1
Name: ticker, dtype: int64

## Merging Data - Concat CSV Files

In [31]:
#Merging all csv files in stockdata
import os
stockdatalist = []
filep = 'StockData/'

#find all csv files
for file in os.listdir(filep):
    if file.endswith(".csv"):
        stockdatalist.append(file)

In [32]:
tables = []
for f in stockdatalist:
    df = pd.read_csv(filep+f, parse_dates=['Date'])
    df['Ticker'] = f.replace('.csv','')
    tables.append(df)

In [33]:
tickerData = pd.concat(tables)
tickerData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 371866 entries, 0 to 4221
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   Date       371866 non-null  datetime64[ns]
 1   Open       371865 non-null  float64       
 2   High       371865 non-null  float64       
 3   Low        371865 non-null  float64       
 4   Close      371865 non-null  float64       
 5   Adj Close  371865 non-null  float64       
 6   Volume     371865 non-null  float64       
 7   Ticker     371866 non-null  object        
dtypes: datetime64[ns](1), float64(6), object(1)
memory usage: 25.5+ MB


## Merging Data - Concat Tabs

In [34]:
allTabs = pd.read_excel('ExData/client_trades.xlsx',sheet_name=None) #imports all tabs
clients = allTabs['Sheet']
clients.columns = ['Client','Client Name']
allTabs.pop('Sheet') #remove just the first sheet

for client in allTabs.keys():
    clientNum = int(client.replace("client_",""))
    table = allTabs[client]
    table['Client'] = clientNum
    allTabs[client] = table

In [35]:
clientsTable = pd.concat(allTabs)
clientsTable['date'] = pd.to_datetime(clientsTable['date'])

In [36]:
clientsTable.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 5849 entries, ('client_1', 0) to ('client_20', 314)
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      5849 non-null   datetime64[ns]
 1   ticker    5849 non-null   object        
 2   buy_sell  5849 non-null   object        
 3   quantity  5849 non-null   int64         
 4   Client    5849 non-null   int64         
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 256.9+ KB


## Merge trades with share prices

In [37]:
mergeData = clientsTable.merge(tickerData, left_on=['ticker','date'],
                   right_on=['Ticker','Date'],how='inner')

mergeData = mergeData.merge(clients, on=['Client'])

In [38]:
mergeData.head()

,date,ticker,buy_sell,quantity,Client,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,Client Name
0,2019-04-02,WHR,buy,100,1,2019-04-02,134.410004,135.149994,133.399994,134.589996,133.354095,563700.0,WHR,CapitalAlley Funds
1,2019-04-02,NEM,buy,100,1,2019-04-02,35.509998,35.919998,35.380001,35.830002,34.815838,11167100.0,NEM,CapitalAlley Funds
2,2019-04-03,GM,sell,100,1,2019-04-03,38.250000,38.639999,38.230000,38.389999,37.982964,8815300.0,GM,CapitalAlley Funds
3,2019-04-04,EA,sell,100,1,2019-04-04,99.989998,100.300003,97.589996,97.940002,97.940002,4308700.0,EA,CapitalAlley Funds
4,2019-04-05,ADBE,sell,100,1,2019-04-05,269.440002,269.940002,266.589996,267.450012,267.450012,1970500.0,ADBE,CapitalAlley Funds


## Merge Approximate Match

In [39]:
#similar to Vlookup with a True in Excel
#useful for merging ranges
feeGrid = pd.read_excel('ExData/FeeGrid.xlsx',skiprows=1)
feeGrid['Share Price'] = feeGrid['Share Price'].astype(float)
allData = pd.merge_asof(mergeData.sort_values('Adj Close'), feeGrid, left_on='Adj Close', right_on='Share Price')
    #"Vlookup with a True" - commissions

#Calculate fees
allData['Fees'] = allData['Fee/Share'] * allData['quantity']
allData['date'] = pd.to_datetime(allData['date'])
allData['$ Volume'] = allData['Adj Close'] * allData['quantity']

In [40]:
allData.head()

,date,ticker,buy_sell,quantity,Client,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,Client Name,Range,Share Price,Fee/Share,Fees,$ Volume
0,2019-06-14,NBL,buy,3300,19,2019-06-14,20.690001,20.870001,19.68,19.73,19.61445,5833100.0,NBL,Second Point Management,$15.00 to $49.99,15.0,0.04,132.0,64727.685
1,2019-06-14,NBL,buy,1600,11,2019-06-14,20.690001,20.870001,19.68,19.73,19.61445,5833100.0,NBL,Beta Dynamic Fund,$15.00 to $49.99,15.0,0.04,64.0,31383.120
2,2019-06-14,NBL,buy,2400,10,2019-06-14,20.690001,20.870001,19.68,19.73,19.61445,5833100.0,NBL,Spectrum Group,$15.00 to $49.99,15.0,0.04,96.0,47074.680
3,2019-06-14,NBL,sell,700,3,2019-06-14,20.690001,20.870001,19.68,19.73,19.61445,5833100.0,NBL,FinanCorp,$15.00 to $49.99,15.0,0.04,28.0,13730.115
4,2019-06-14,NBL,sell,600,8,2019-06-14,20.690001,20.870001,19.68,19.73,19.61445,5833100.0,NBL,KKZ Asset Management,$15.00 to $49.99,15.0,0.04,24.0,11768.670


## Merge Sector Data

In [41]:
sectorsData = pd.read_excel('ExData/S&P500.xlsx')
allData = allData.merge(sectorsData, left_on=['Ticker'], right_on=['Symbol'])
#allData.to_excel('Output/allData.xlsx')

In [42]:
allData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5849 entries, 0 to 5848
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   date                   5849 non-null   datetime64[ns]
 1   ticker                 5849 non-null   object        
 2   buy_sell               5849 non-null   object        
 3   quantity               5849 non-null   int64         
 4   Client                 5849 non-null   int64         
 5   Date                   5849 non-null   datetime64[ns]
 6   Open                   5849 non-null   float64       
 7   High                   5849 non-null   float64       
 8   Low                    5849 non-null   float64       
 9   Close                  5849 non-null   float64       
 10  Adj Close              5849 non-null   float64       
 11  Volume                 5849 non-null   float64       
 12  Ticker                 5849 non-null   object        
 13  Cli

In [43]:
allData.head()

,date,ticker,buy_sell,quantity,Client,Date,Open,High,Low,Close,...,Fees,$ Volume,Symbol,Company Name,GICS Sector,GICS Sub Industry,Headquarters Location,Date first added,CIK,Founded
0,2019-06-14,NBL,buy,3300,19,2019-06-14,20.690001,20.870001,19.68,19.73,...,132.0,64727.685,NBL,Noble Energy Inc,Energy,Oil & Gas Exploration & Production,"Houston, Texas",2007-10-08 00:00:00,72207,NaN
1,2019-06-14,NBL,buy,1600,11,2019-06-14,20.690001,20.870001,19.68,19.73,...,64.0,31383.120,NBL,Noble Energy Inc,Energy,Oil & Gas Exploration & Production,"Houston, Texas",2007-10-08 00:00:00,72207,NaN
2,2019-06-14,NBL,buy,2400,10,2019-06-14,20.690001,20.870001,19.68,19.73,...,96.0,47074.680,NBL,Noble Energy Inc,Energy,Oil & Gas Exploration & Production,"Houston, Texas",2007-10-08 00:00:00,72207,NaN
3,2019-06-14,NBL,sell,700,3,2019-06-14,20.690001,20.870001,19.68,19.73,...,28.0,13730.115,NBL,Noble Energy Inc,Energy,Oil & Gas Exploration & Production,"Houston, Texas",2007-10-08 00:00:00,72207,NaN
4,2019-06-14,NBL,sell,600,8,2019-06-14,20.690001,20.870001,19.68,19.73,...,24.0,11768.670,NBL,Noble Energy Inc,Energy,Oil & Gas Exploration & Production,"Houston, Texas",2007-10-08 00:00:00,72207,NaN


## Filtering Data

In [44]:
cond1 = allData['ticker'] == 'AAPL'
cond2 = allData['buy_sell'] == 'buy'
filter1 = allData[cond1 & cond2]
filter1.head()

,date,ticker,buy_sell,quantity,Client,Date,Open,High,Low,Close,...,Fees,$ Volume,Symbol,Company Name,GICS Sector,GICS Sub Industry,Headquarters Location,Date first added,CIK,Founded
4894,2019-06-03,AAPL,buy,1200,13,2019-06-03,175.600006,177.919998,170.270004,173.300003,...,150.0,207960.0036,AAPL,Apple Inc.,Information Technology,"Technology Hardware, Storage & Peripherals","Cupertino, California",1982-11-30 00:00:00,320193,1977
4895,2019-06-03,AAPL,buy,1200,9,2019-06-03,175.600006,177.919998,170.270004,173.300003,...,150.0,207960.0036,AAPL,Apple Inc.,Information Technology,"Technology Hardware, Storage & Peripherals","Cupertino, California",1982-11-30 00:00:00,320193,1977
4896,2019-05-31,AAPL,buy,700,9,2019-05-31,176.229996,177.990005,174.990005,175.070007,...,87.5,122549.0049,AAPL,Apple Inc.,Information Technology,"Technology Hardware, Storage & Peripherals","Cupertino, California",1982-11-30 00:00:00,320193,1977
4898,2019-05-31,AAPL,buy,800,20,2019-05-31,176.229996,177.990005,174.990005,175.070007,...,100.0,140056.0056,AAPL,Apple Inc.,Information Technology,"Technology Hardware, Storage & Peripherals","Cupertino, California",1982-11-30 00:00:00,320193,1977
4899,2019-05-29,AAPL,buy,200,8,2019-05-29,176.419998,179.350006,176.000000,177.380005,...,25.0,35476.0010,AAPL,Apple Inc.,Information Technology,"Technology Hardware, Storage & Peripherals","Cupertino, California",1982-11-30 00:00:00,320193,1977


In [45]:
cond1 = allData['Client Name'] == 'Marquee Group Funds'
cond2 = allData['GICS Sector'] == 'Materials'
cond3 = allData['GICS Sector'] == 'Energy'
filter2 = allData[cond1 & (cond2 | cond3)]
filter2

,date,ticker,buy_sell,quantity,Client,Date,Open,High,Low,Close,...,Fees,$ Volume,Symbol,Company Name,GICS Sector,GICS Sub Industry,Headquarters Location,Date first added,CIK,Founded
26,2019-06-04,NBL,buy,1600,13,2019-06-04,21.860001,21.910000,21.340000,21.510000,...,64.0,34214.4432,NBL,Noble Energy Inc,Energy,Oil & Gas Exploration & Production,"Houston, Texas",2007-10-08 00:00:00,72207,NaN
49,2019-06-26,NBL,buy,900,13,2019-06-26,21.549999,22.240000,21.420000,21.770000,...,36.0,19478.2536,NBL,Noble Energy Inc,Energy,Oil & Gas Exploration & Production,"Houston, Texas",2007-10-08 00:00:00,72207,NaN
56,2019-05-10,NBL,sell,2400,13,2019-05-10,24.270000,24.330000,23.570000,24.110001,...,96.0,57525.1200,NBL,Noble Energy Inc,Energy,Oil & Gas Exploration & Production,"Houston, Texas",2007-10-08 00:00:00,72207,NaN
61,2019-05-21,NBL,buy,1700,13,2019-05-21,24.090000,24.650000,24.080000,24.270000,...,68.0,41017.3671,NBL,Noble Energy Inc,Energy,Oil & Gas Exploration & Production,"Houston, Texas",2007-10-08 00:00:00,72207,NaN
83,2019-04-08,NBL,sell,1100,13,2019-04-08,25.760000,26.010000,25.320000,25.469999,...,44.0,27721.7424,NBL,Noble Energy Inc,Energy,Oil & Gas Exploration & Production,"Houston, Texas",2007-10-08 00:00:00,72207,NaN
108,2019-04-29,NBL,sell,900,13,2019-04-29,27.080000,27.670000,27.080000,27.420000,...,36.0,24417.9297,NBL,Noble Energy Inc,Energy,Oil & Gas Exploration & Production,"Houston, Texas",2007-10-08 00:00:00,72207,NaN
603,2019-04-26,NEM,buy,1300,13,2019-04-26,31.870001,31.940001,31.370001,31.459999,...,52.0,40731.3608,NEM,Newmont Goldcorp,Materials,Gold,"Denver, Colorado",1969-06-30 00:00:00,1164727,1921
606,2019-04-25,NEM,sell,900,13,2019-04-25,32.830002,32.830002,31.170000,31.629999,...,36.0,28351.0098,NEM,Newmont Goldcorp,Materials,Gold,"Denver, Colorado",1969-06-30 00:00:00,1164727,1921
636,2019-04-02,NEM,sell,1800,13,2019-04-02,35.509998,35.919998,35.380001,35.830002,...,72.0,62668.5084,NEM,Newmont Goldcorp,Materials,Gold,"Denver, Colorado",1969-06-30 00:00:00,1164727,1921
641,2019-04-03,NEM,sell,1000,13,2019-04-03,35.869999,36.099998,35.709999,35.950001,...,40.0,34932.4380,NEM,Newmont Goldcorp,Materials,Gold,"Denver, Colorado",1969-06-30 00:00:00,1164727,1921
